# Churn Clustering

In [ ]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.clustering import KMeans, BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from helpers.helper_functions import translate_to_file_string

## Select the churn file 

In [ ]:
inputFile = translate_to_file_string("../data/churn.csv")

## Create the Spark Session 

In [ ]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("ChurnClustering")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   

## Data Preparation
### Transform labels into index

In [ ]:
labelIndexer = StringIndexer().setInputCol("LEAVE").setOutputCol("label").fit(df)
collegeIndexer = StringIndexer().setInputCol("COLLEGE").setOutputCol("COLLEGE_NUM").fit(df)
satIndexer = StringIndexer().setInputCol("REPORTED_SATISFACTION").setOutputCol("REPORTED_SATISFACTION_NUM").fit(df)
usageIndexer = StringIndexer().setInputCol("REPORTED_USAGE_LEVEL").setOutputCol("REPORTED_USAGE_LEVEL_NUM").fit(df)
changeIndexer = StringIndexer().setInputCol("CONSIDERING_CHANGE_OF_PLAN").setOutputCol("CONSIDERING_CHANGE_OF_PLAN_NUM").fit(df)

 ### Build the feature vector

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("LEAVE")
featureCols.remove("COLLEGE")
featureCols.remove("REPORTED_SATISFACTION")
featureCols.remove("REPORTED_USAGE_LEVEL")
featureCols.remove("CONSIDERING_CHANGE_OF_PLAN")
featureCols = featureCols +["COLLEGE_NUM","REPORTED_SATISFACTION_NUM","REPORTED_USAGE_LEVEL_NUM","CONSIDERING_CHANGE_OF_PLAN_NUM"]

### Build the feature Vector Assembler

In [ ]:
assembler =  VectorAssembler(outputCol="features", inputCols=list(featureCols))

### Build a featureIndexer 

Automatically identify categorical features, and index them.
Features with > 5 distinct values are treated as continuous.

In [ ]:
featureIndexer = VectorIndexer(inputCol="features",outputCol="indexedFeatures", maxCategories=6) 

## Build KMeans

In [ ]:
#TODO specify paramams accordingly
km = KMeans (seed=12345, featuresCol="indexedFeatures",predictionCol="prediction")   

## Data Preparation
### Build a network parameters grid

In [ ]:
# TODO optimize params and add appropriate ones
paramGrid = ParamGridBuilder().addGrid(km.k, [2]) \
				                 .addGrid(km.maxIter, [10]) \
                                 .build()

### Build a pipeline

In [ ]:
pipeline = Pipeline(stages = [labelIndexer, collegeIndexer, satIndexer,
				usageIndexer, changeIndexer, assembler, featureIndexer, km ])

## Build an evaluator

In [ ]:
evaluator = ClusteringEvaluator()

## Build the Cross Validator

In [ ]:
 # TODO change to usefull settings
 cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=2, parallelism=2)

## Train the Model 

In [ ]:
cvModel = cv.fit(df)

## Find out what is the best model

In [ ]:
kmModel = cvModel.bestModel.stages[7]
print(kmModel.explainParams())
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

## Test the model 

In [ ]:
predictions = cvModel.transform(df)
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " , silhouette)

## Bisecting k-means

Build the Bisecting KMeans

In [ ]:
# TODO add usefull params 
bkm = BisectingKMeans(k=2, seed=12345, featuresCol="indexedFeatures", predictionCol="prediction")

Param Grid for Bisecting

In [ ]:
# TODO change params and add appropriate ones
paramGridBkm = ParamGridBuilder().addGrid(bkm.k, [ 2]) \
				                .addGrid(bkm.maxIter, [10]) \
                                .build()

Pipeline for Bisecting

In [ ]:
pipelineBkm = Pipeline(stages = [labelIndexer, collegeIndexer, satIndexer,
				usageIndexer, changeIndexer, assembler, featureIndexer, bkm ])

Build the cross validation

In [ ]:
# TODO change to usefull setting
cvbkm = CrossValidator(estimator=pipelineBkm,evaluator=evaluator,estimatorParamMaps=paramGridBkm, numFolds=2,parallelism=2)

## Train the Model

In [ ]:
cvModebkml = cvbkm.fit(df)

### Find the best model

In [ ]:
kmModelbkm = cvModebkml.bestModel.stages[7]
print(kmModelbkm.explainParams())
centers = kmModelbkm.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

### Test the model

In [ ]:
predictions = cvModebkml.transform(df)
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " , silhouette)

In [ ]:
spark.stop()